In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500000)
pd.set_option('display.max_columns', 1000000)
pd.set_option('display.width', 100000)

In [2]:
#importing datasets
tweets=pd.read_csv('tweets.csv')
tweets.drop(['Tweet_processed'], axis = 1, inplace=True)
tweets['Date']=pd.to_datetime(tweets['Date'])

In [3]:
tweets.dtypes

Date     datetime64[ns]
Tweet            object
dtype: object

In [4]:
IBM=pd.read_csv('IBM_clean.csv')
IBM['Date']=pd.to_datetime(IBM['Date'])
IBM.tail()

,Date,Open,High,Low,Close,Volume
14582,2019-12-05,132.990005,133.240005,131.630005,131.910004,3595400
14583,2019-12-06,132.750000,133.889999,132.699997,133.220001,3437500
14584,2019-12-09,133.350006,134.589996,133.259995,133.919998,2968500
14585,2019-12-10,133.919998,134.839996,133.559998,133.910004,3480400
14586,2019-12-11,134.110001,134.509995,133.669998,133.759995,3953300


In [5]:
IBM.dtypes

Date      datetime64[ns]
Open             float64
High             float64
Low              float64
Close            float64
Volume             int64
dtype: object

In [6]:
merged = IBM.merge(tweets, on='Date', how = 'outer')

In [8]:
merged.head()

,Date,Open,High,Low,Close,Volume,Tweet
0,1962-01-02,7.713333,7.713333,7.626667,7.626667,387200.0,NaN
1,1962-01-03,7.626667,7.693333,7.626667,7.693333,288000.0,NaN
2,1962-01-04,7.693333,7.693333,7.613333,7.616667,256000.0,NaN
3,1962-01-05,7.606667,7.606667,7.453333,7.466667,363200.0,NaN
4,1962-01-08,7.460000,7.460000,7.266667,7.326667,544000.0,NaN


In [9]:
merged.shape

(14864, 7)

In [10]:
# Cleaning rows that are duplicated
merged = merged.drop(merged.index[np.where(merged.index >= 14587)[0]])

In [11]:
# Filling missing tweets with a space
merged['Tweet'] = merged['Tweet'].replace(np.nan, '', regex=True)

In [12]:
merged.shape

(14587, 7)

In [13]:
merged['compound']=''
merged['neg']=''
merged['neu']=''
merged['pos']=''

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [15]:
sia=SentimentIntensityAnalyzer()

In [16]:
for index,sentence in enumerate(merged['Tweet']):
    ps = sia.polarity_scores(sentence)
    
    merged['compound'][index] = ps['compound']
    merged['neg'][index] = ps['neg']
    merged['neu'][index] = ps['neu']
    merged['pos'][index] = ps['pos']


/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Applications/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Applications/anaconda3/lib/py

In [17]:
# removing tweets
merged.drop('Tweet', axis=1, inplace=True)

In [18]:
#converting types
merged.dtypes

Date        datetime64[ns]
Open               float64
High               float64
Low                float64
Close              float64
Volume             float64
compound            object
neg                 object
neu                 object
pos                 object
dtype: object

In [19]:
merged['neu'] = pd.to_numeric(merged['neu'])
merged['pos'] = pd.to_numeric(merged['pos'])
merged['neg'] = pd.to_numeric(merged['neg'])
merged['compound'] = pd.to_numeric(merged['compound'])

In [20]:
merged.dtypes

Date        datetime64[ns]
Open               float64
High               float64
Low                float64
Close              float64
Volume             float64
compound           float64
neg                float64
neu                float64
pos                float64
dtype: object

In [22]:
merged.tail()

,Date,Open,High,Low,Close,Volume,compound,neg,neu,pos
14582,2019-12-05,132.990005,133.240005,131.630005,131.910004,3595400.0,0.3983,0.057,0.813,0.130
14583,2019-12-06,132.750000,133.889999,132.699997,133.220001,3437500.0,0.9939,0.055,0.607,0.338
14584,2019-12-09,133.350006,134.589996,133.259995,133.919998,2968500.0,0.9955,0.000,0.620,0.380
14585,2019-12-10,133.919998,134.839996,133.559998,133.910004,3480400.0,0.9183,0.000,0.791,0.209
14586,2019-12-11,134.110001,134.509995,133.669998,133.759995,3953300.0,0.9508,0.037,0.706,0.257


In [23]:
# exporting data to csv file
merged.to_csv('ml_data.csv',index=False)